In [1]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import utils, models,datasets
import torchvision.transforms as transforms
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import time



/home/asineesh/I2I/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class block(nn.Module):
    def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super().__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels,intermediate_channels,kernel_size=1, stride=1, padding=0,bias=False)
        self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels,kernel_size=3, stride=stride,padding=1, bias=False)
        self.conv3 = nn.Conv2d(intermediate_channels,intermediate_channels * self.expansion,kernel_size=1,stride=1,padding=0,bias=False)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()
        x = self.conv1(x)        
        x = self.relu(x)
        x = self.conv2(x)        
        x = self.relu(x)
        x = self.conv3(x)       

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Conv2d(self.in_channels,intermediate_channels * 4,kernel_size=1,stride=stride,bias=False)               

        layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride))

        self.in_channels = intermediate_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [3]:
train_transform = transforms.Compose([transforms.Resize((96,96)),
                                      transforms.ColorJitter(brightness=.5, saturation=.25,hue=.1, contrast=.5),
                                      transforms.RandomAffine(10, (0.05, 0.05)),
                                      transforms.RandomHorizontalFlip(.5),
                                      transforms.RandomVerticalFlip(.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.6716241, 0.48636872, 0.60884315],
                                                           [0.27210504, 0.31001145, 0.2918652])
        ])

test_transform = transforms.Compose([transforms.Resize((96,96)),        
            transforms.ToTensor(),
            transforms.Normalize([0.6716241, 0.48636872, 0.60884315],
                                 [0.27210504, 0.31001145, 0.2918652])
        ])

In [4]:
train_data = datasets.ImageFolder(root="PCam/Pcam_Train/Pcam_Train",transform=train_transform)
len(train_data)

245760

In [5]:
valid_data = datasets.ImageFolder(root="PCam/Pcam_Test_192/Pcam_Test_192",transform=test_transform)
len(valid_data)

40960

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
model = ResNet(block, [3, 4, 6, 3], 3, 2)
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True,num_workers=24)
test_loader = DataLoader(valid_data, batch_size=128, shuffle=True,num_workers=24)
torch.backends.cudnn.benchmark = False

In [9]:
start_time = time.time()     
    
train_losses = []
train_correct = []
test_correct = []
best_acc = 0.0
    
for epoch in range(10):
        trn_corr = 0
        tst_corr = 0
        loop = tqdm(train_loader)
                
        # Run the training batches
        for b, (X_train, y_train) in enumerate(loop):       
            
            X_train, y_train = X_train.to(device), y_train.to(device)
            y_pred = model(X_train)
            loss = criterion(y_pred, y_train)
     
            # Tally the number of correct predictions
            predicted = torch.max(y_pred.data, 1)[1]
            batch_corr = (predicted == y_train).sum()
            trn_corr += batch_corr
            
            # Update parameters
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())   
           
    
            train_losses.append(loss)
            train_correct.append(trn_corr)
        
        print(f'epoch: {epoch:2} Train accuracy: {train_correct[-1].item()*100/len(train_data):.3f}%')
        
        # Run the testing batches
        
        with torch.no_grad():
            for b, (X_test, y_test) in enumerate(test_loader):
                
                X_test, y_test = X_test.to(device), y_test.to(device)
                
                # Apply the model
                
                y_val = model(X_test)              
                
                # Tally the number of correct predictions
                predicted = torch.max(y_val.data, 1)[1] 
                tst_corr += (predicted == y_test).sum()
                test_correct.append(tst_corr)               
        
        
        print(f'epoch: {epoch:2} Test accuracy: {test_correct[-1].item()*100/len(valid_data):.3f}%')
        test_acc = test_correct[-1].item()*100/len(valid_data)
        if test_acc>best_acc:
            torch.save(model.state_dict(), "resnet96_0.0005_"+str(test_acc)+"_"+str(epoch)+" .pt")
            best_acc = test_acc
       
        print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed    
    
print(f'Final Test accuracy: {test_correct[-1].item()*100/len(valid_data):.3f}%')

100%|██████████| 1920/1920 [08:28<00:00,  3.78it/s, loss=0.446]

epoch:  0 Train accuracy: 75.667%


epoch:  0 Test accuracy: 79.419%

Duration: 554 seconds


100%|██████████| 1920/1920 [08:09<00:00,  3.93it/s, loss=0.499]

epoch:  1 Train accuracy: 80.142%


...

epoch:  8 Train accuracy: 89.080%


epoch:  8 Test accuracy: 86.965%

Duration: 4824 seconds


100%|██████████| 1920/1920 [08:01<00:00,  3.99it/s, loss=0.254]

epoch:  9 Train accuracy: 89.594%


epoch:  9 Test accuracy: 85.908%

Duration: 5350 seconds
Final Test accuracy: 85.908%
